In [50]:
#This pandas library helps to create data frame in the program
import pandas as pd

#This request library helps to fetch the json data from url and process it
import requests as req

#ListUnwrapper class helps in flattening nested list data inside the JSON Data
class ListUnwrapper:
    
    #This is the constructor of ListUnwrapper Class
    def __init__(self):
        self._data_list = None
    
    #This is the getter method to fetch the data from protected _data_list variable
    def get_data_list(self):
        return self._data_list
    
    #This is the setter method to set the data into protected _data_list variable
    def set_data_list(self, data_list):
        self._data_list = data_list
    
    #This static method unwrap_list does the unwrapping for nested list inside the JSON data
    @staticmethod
    def unwrap_list(data_series, col):
        data_list = []
        for data in data_series:
            data_dict = {}
            if data is not None:
                if(type(data) == list and len(data) >= 1):
                    for i in range(len(data)):
                        data_dict[str(col)+"."+str(i)] = data[i]
                        if type(data[i]) == dict:
                            data_dict.update((DictUnwrapper.unwrap_dict([data[i]], str(col)+"."+str(i)))[0])
                data_list.append(data_dict)
        unwrap_list_obj = ListUnwrapper()
        unwrap_list_obj.set_data_list(data_list)
        return unwrap_list_obj.get_data_list()


#DictUnwrapper class helps in flattening nested dictionary data inside the JSON Data
class DictUnwrapper:
    
    #This is the constructor of DictUnwrapper Class
    def __init__(self):
        self._data_list = None
    
    #This is the getter method to fetch the data from protected _data_list variable
    def get_data_list(self):
        return self._data_list
    
    #This is the setter method to set the data into protected _data_list variable
    def set_data_list(self, data_list):
        self._data_list = data_list
    
    #This static method unwrap_dict does the unwrapping for nested dictionary inside the JSON data
    @staticmethod
    def unwrap_dict(data_series, col):
        data_list = []
        for data in data_series:
            data_dict = {}
            if data is not None:
                if(type(data) == dict):
                    for key, values in data.items():
                        data_dict[str(col)+"."+str(key)] = values
                        if type(values) == dict:
                            data_dict.update((DictUnwrapper.unwrap_dict([values], str(col)+"."+str(key)))[0])
                data_list.append(data_dict)
        unwrap_dict_obj = DictUnwrapper()
        unwrap_dict_obj.set_data_list(data_list)
        return unwrap_dict_obj.get_data_list()


#JSONUnwrapper class helps in flattening the overall JSON data fed to it and return the final result.
class JSONUnwrapper:
    
    #This is the constructor of JSONUnwrapper Class
    def __init__(self, url):
        self._url = url
        self._json_data_frame = None
    
    #This is the getter method to fetch the data from protected _url variable
    def get_url(self):
        return self._url
    
    #This is the setter method to set the data into protected _url variable
    def set_url(self, url):
        self._url = url
    
    #This is the getter method to fetch the data from protected _json_data_frame variable
    def get_json_data_frame(self):
        return self._json_data_frame
    
    #This is the setter method to set the data into protected _json_data_frame variable
    def set_json_data_frame(self, json_data_frame):
        self._json_data_frame = json_data_frame
    
    #This get_json_data method helps to fetch the JSON data from class JSONExtractor
    def get_json_data(self):
        json_obj = JSONExtractor(self.get_url())
        return json_obj.get_json_data()
    
    #This json_unwrapper is the primary method of this class which unwraps the whole JSON data and converts it into data frame and returns the result.
    def json_unwrapper(self):
        
        try:
            
            json_data_frame = pd.DataFrame(self.get_json_data())
            frame_list = []
            frame_list.append(json_data_frame)
            for col in json_data_frame.keys():
                for data in json_data_frame[col]:
                    if data is not None:
                        if(type(data) == dict):
                            temp_frame = pd.DataFrame(DictUnwrapper.unwrap_dict(json_data_frame[col], col))
                            frame_list.append(temp_frame)
                            break
                        elif(type(data) == list):
                            temp_frame = pd.DataFrame(ListUnwrapper.unwrap_list(json_data_frame[col], col))
                            frame_list.append(temp_frame)
                            break
            self.set_json_data_frame(pd.concat(frame_list, axis=1)) #pd.concat() is used to merge multiple data frames sequentially.
            return self.get_json_data_frame().fillna("None") #fillna() is used to replace the NaN values in your whole data frame with a value of your choice
            
        except Exception as e:
            
            print("Error occured due to: {} ".format(str(e)))


#JSONExtractor class is used to extract the JSON data from the URL given as input and return the JSON data to JSONUnwrapper class
class JSONExtractor:
    
    #This is the constructor of JSONExtractor Class
    def __init__(self, url):
        self._url = url
        self._json_data = None
    
    #This is the getter method to fetch the data from protected _url variable
    def get_url(self):
        return self._url
    
    #This is the setter method to set the data into protected _json_data variable
    def set_json_data(self, json_resp):
        self._json_data = json_resp
    
    #This is the getter method to fetch the data from protected _url variable, creates a json response and returns it.
    def get_json_data(self):
        
        try:
            
            resp = req.get(self.get_url())
            self.set_json_data(resp.json())
            return self._json_data
            
        except Exception as e:
            
            print("Error occured due to: {} ".format(str(e)))
            

#input
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
result_obj = JSONUnwrapper(url)
result_obj.json_unwrapper()
#result_obj.json_unwrapper().to_csv("Unravelled_DataSet.csv")

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,milestone.closed_issues,milestone.state,milestone.created_at,milestone.updated_at,milestone.due_on,milestone.closed_at,pull_request.url,pull_request.html_url,pull_request.diff_url,pull_request.patch_url
0,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36240,696387789,MDExOlB1bGxSZXF1ZXN0NDgyNDc1OTk4,36240,PERF: Allow groupby transform with numba engin...,...,330,open,2020-05-29T23:47:32Z,2020-09-09T04:04:43Z,2020-11-29T08:00:00Z,None,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36240,https://github.com/pandas-dev/pandas/pull/3624...,https://github.com/pandas-dev/pandas/pull/3624...
1,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36239,696372190,MDExOlB1bGxSZXF1ZXN0NDgyNDYyMzUx,36239,Update base.py,...,330,open,2020-05-29T23:47:32Z,2020-09-09T04:04:43Z,2020-11-29T08:00:00Z,None,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36239,https://github.com/pandas-dev/pandas/pull/3623...,https://github.com/pandas-dev/pandas/pull/3623...
2,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36238,696358898,MDExOlB1bGxSZXF1ZXN0NDgyNDUwNTg0,36238,Update groupby.rst,...,330,open,2020-05-29T23:47:32Z,2020-09-09T04:04:43Z,2020-11-29T08:00:00Z,None,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36238,https://github.com/pandas-dev/pandas/pull/3623...,https://github.com/pandas-dev/pandas/pull/3623...
3,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36237,696348041,MDExOlB1bGxSZXF1ZXN0NDgyNDQxMjQx,36237,CLN: simplify Categorical comparisons,...,3,open,2016-08-16T13:21:24Z,2020-09-08T11:18:17Z,2021-10-01T07:00:00Z,None,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36237,https://github.com/pandas-dev/pandas/pull/3623...,https://github.com/pandas-dev/pandas/pull/3623...
4,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36236,696281247,MDExOlB1bGxSZXF1ZXN0NDgyMzg0MDE5,36236,TST: add test case for sort_index on multiinde...,...,330,open,2020-05-29T23:47:32Z,2020-09-09T04:04:43Z,2020-11-29T08:00:00Z,None,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36236,https://github.com/pandas-dev/pandas/pull/3623...,https://github.com/pandas-dev/pandas/pull/3623...
5,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://api.github.com/repos/pandas-dev/pandas...,https://github.com/pandas-dev/pandas/pull/36233,696246890,MDExOlB1bGxSZXF1ZXN0NDgyMzU3MjEw,36233,TST/DOC: Fixed #31771,...,606,open,2015-01-13T10:53:19Z,2020-09-